In [ ]:
import os
import numpy as np
from minimodel import data

# load image and neural data

In [ ]:
# load image
mouse_id = 5

# load images
data_path = './'
img = data.load_images(data_path, mouse_id, file=os.path.join(data_path, data.img_file_name[mouse_id]), normalize=False)
print('img: ', img.shape, img.min(), img.max(), img.dtype)

# load neurons
fname = '%s_nat60k_%s.npz'%(data.db[mouse_id]['mname'], data.db[mouse_id]['datexp'])
spks, istim_train, istim_test, xpos, ypos, spks_rep_all = data.load_neurons(file_path = os.path.join(data_path, fname), mouse_id = mouse_id)
n_stim, n_max_neurons = spks.shape

# split train and validation set
itrain, ival = data.split_train_val(istim_train, train_frac=0.9)

ineur = np.arange(0, n_max_neurons)
spks_train = spks[itrain][:,ineur]
spks_val = spks[ival][:,ineur]

print('spks_train: ', spks_train.shape, spks_train.min(), spks_train.max())
print('spks_val: ', spks_val.shape, spks_val.min(), spks_val.max())

img_train = img[istim_train][itrain]
img_val = img[istim_train][ival]
img_test = img[istim_test]

print('img_train: ', img_train.shape, img_train.min(), img_train.max())
print('img_val: ', img_val.shape, img_val.min(), img_val.max())
print('img_test: ', img_test.shape, img_test.min(), img_test.max())

input_Ly, input_Lx = img_train.shape[-2:]

In [ ]:
# Initialize lists to store reshaped spks_test and img_test
spks_test_list = []
img_test_list = []
image_id_test_list = []

# Iterate over each stimulus in the test set
for i, spks in enumerate(spks_rep_all):
    nrep, nneurons = spks.shape
    spks_test_list.append(spks)
    
    # Reshape the corresponding img_test
    img_rep = img_test[i][None, :, :].repeat(nrep, axis=0)
    img_test_list.append(img_rep)
    
    # Repeat the image ID for each repeat
    image_id_test_list.append(np.repeat(istim_test[i], nrep))

# Concatenate the reshaped test data
spks_test = np.concatenate(spks_test_list, axis=0)
img_test = np.concatenate(img_test_list, axis=0)
image_id_test = np.concatenate(image_id_test_list, axis=0)

print('spks_test: ', spks_test.shape, spks_test.min(), spks_test.max())
print('img_test: ', img_test.shape, img_test.min(), img_test.max())
print('image_id_test: ', image_id_test.shape, image_id_test.min(), image_id_test.max())

# Concatenate train, val, and test sets
spk_all = np.concatenate([spks_train, spks_val, spks_test], axis=0)
img_all = np.concatenate([img_train, img_val, img_test], axis=0)
image_id_all = np.concatenate([istim_train[itrain], istim_train[ival], image_id_test], axis=0)

ntrain, nval = len(spks_train), len(spks_val)
ntest = len(spks_test)
itrain = np.arange(ntrain)
ival = np.arange(ntrain, ntrain + nval)
itest = np.arange(ntrain + nval, ntrain + nval + ntest)

print('spks_all: ', spk_all.shape, spk_all.min(), spk_all.max())
print('img_all: ', img_all.shape, img_all.min(), img_all.max())
print('image_id_all: ', image_id_all.shape, image_id_all.min(), image_id_all.max())

In [ ]:
# save image and neural data
print(spk_all.shape, img_all.shape)
NT, NN = spk_all.shape
data_path = f'./nat30k_{data.mouse_names[mouse_id]}_{data.exp_date[mouse_id]}'
os.makedirs(data_path, exist_ok=True)

# set up data folder
img_path = os.path.join(data_path, 'data/images')
spk_path = os.path.join(data_path, 'data/responses')
os.makedirs(img_path, exist_ok=True)
os.makedirs(spk_path, exist_ok=True)

for i in range(img_all.shape[0]):
    img_savepath = os.path.join(img_path, f'{i}.npy')
    spk_savepath = os.path.join(spk_path, f'{i}.npy')
    timg = img_all[i]
    np.save(img_savepath, timg[np.newaxis, ...])
    np.save(spk_savepath, spk_all[i])

In [ ]:
# setup meta folder
# setup meta/statistics folder
img_stats = [img_all.max(), img_all.min(), img_all.mean(), np.median(img[:]), img.std()]
img_stats_folder = os.path.join(data_path, 'meta', 'statistics', 'images')
os.makedirs(img_stats_folder, exist_ok=True)

X_stats = [spk_all.max(axis=0), spk_all.min(axis=0), spk_all.mean(axis=0), np.median(spk_all, axis=0), spk_all.std(axis=0)]
X_stats_folder = os.path.join(data_path, 'meta', 'statistics', 'responses')
os.makedirs(X_stats_folder, exist_ok=True)

stats_names = ['max', 'min', 'mean', 'median', 'std']

for folder in ['all', 'stimulus_frame']:
    Xs_folder = os.path.join(X_stats_folder, folder)
    os.makedirs(Xs_folder, exist_ok=True)
    imgs_folder = os.path.join(img_stats_folder, folder)
    os.makedirs(imgs_folder, exist_ok=True)
    for istat, xstat, sname in zip(img_stats, X_stats, stats_names):
        np.save(os.path.join(Xs_folder, f'{sname}.npy'), xstat)
        np.save(os.path.join(imgs_folder, f'{sname}.npy'), istat)

In [ ]:
# setup meta/trials folder
# save tiers.npy
meta_trials_folder = os.path.join(data_path, 'meta', 'trials')
os.makedirs(meta_trials_folder, exist_ok=True)
tier_path = os.path.join(meta_trials_folder, 'tiers.npy')

tiers = np.zeros(NT, object)
tiers[itrain] = 'train'
tiers[ival] = 'validation'
tiers[itest] = 'test'
tiers = np.array(list(tiers))
np.save(tier_path, tiers)

# save frame_image_id.npy
frame_image_id_path = os.path.join(meta_trials_folder, 'frame_image_id.npy')
np.save(frame_image_id_path, image_id_all)

# save trial_idx.npy
trial_idx_path = os.path.join(meta_trials_folder, 'trial_idx.npy')
trial_idx = np.arange(NT)
np.save(trial_idx_path, trial_idx)

# save frame_image_class.npy
image_class_path = os.path.join(meta_trials_folder, 'frame_image_class.npy')
image_class = np.zeros(NT)
np.save(image_class_path, image_class)

In [ ]:
# meta/neurons folder
meta_neurons_folder = os.path.join(data_path, 'meta', 'neurons')
os.makedirs(meta_neurons_folder, exist_ok=True)
# sen_meta_neurons_folder = os.path.join(sen_data_path, 'meta', 'neurons')
# NN_sen = len(np.load(os.path.join(sen_meta_neurons_folder, 'layer.npy')))

# save cell_motor_coordinates.npy
coordinates_path = os.path.join(meta_neurons_folder, 'cell_motor_coordinates.npy')
data = np.zeros((NN, 3))
data[:, 0] = xpos
data[:, 1] = ypos
np.save(coordinates_path, data)

# save unit_ids.npy
unit_ids_path = os.path.join(meta_neurons_folder, 'unit_ids.npy')
unit_ids = np.arange(NN)
np.save(unit_ids_path, unit_ids)

# save area.npy
area_path = os.path.join(meta_neurons_folder, 'area.npy')
area = np.array(['V1'] * NN)
np.save(area_path, area)

# save animal_ids.npy
animal_ids_path = os.path.join(meta_neurons_folder, 'animal_ids.npy')
animal_ids = np.ones(NN) * mouse_id
np.save(animal_ids_path, animal_ids.astype(int))

# save sessions.npy
session_path = os.path.join(meta_neurons_folder, 'sessions.npy')
sessions = np.ones(NT)
np.save(session_path, sessions.astype(int))

# save scan_idx.npy
scan_idx_path = os.path.join(meta_neurons_folder, 'scan_idx.npy')
scan_idx = np.ones(NT)
np.save(scan_idx_path, scan_idx.astype(int))

# check dataloader

In [ ]:
# loading the SENSORIUM dataset
from nnfabrik.builder import get_data
dataset_name = 'nat30k'
n_layers = 1
filenames = [data_path, ]
dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':1.0,
                 'add_behavior_as_channels': False,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [ ]:
dataloaders

In [ ]:
tier = 'test'
sen_dataset_name = f'{mouse_id}-1-1' # animal_id-session_id-scan_idx

images, responses = [], []
for x, y in dataloaders[tier][sen_dataset_name]:
    images.append(x.squeeze().cpu().data.numpy())
    responses.append(y.squeeze().cpu().data.numpy())
    
images = np.vstack(images)
responses = np.vstack(responses)

print('The \"{}\" set of dataset \"{}\" contains the responses of {} neurons to {} images'.format(tier, dataset_name, responses.shape[1], responses.shape[0]))